In [12]:
import LinearAlgebra

struct Point{}
    x::Real
    y::Real
end

dist(p::Point) = sqrt(p.x^2 + p.y^2)
dist(p1::Point, p2::Point) = sqrt((p1.x-p2.x)^2 + (p1.y-p2.y)^2)

LinearAlgebra.norm(p::Point) = dist(p)
LinearAlgebra.dot(p1::Point, p2::Point) = p1.x * p2.x + p1.y * p2.y

Base.:+(p1::Point, p2::Point) = Point(p1.x + p2.x, p1.y + p2.y)
Base.:-(p1::Point, p2::Point) = Point(p1.x - p2.x, p1.y - p2.y)
Base.:-(p::Point) = Point(-p.x,-p.y)
Base.:*(α::Number, p::Point) = Point(α * p.x, α * p.y)
Base.:*(p::Point, α::Number) = α * p
Base.:/(p::Point, α::Number) = p * (1/α)

"""
    center(points) -> Point

Центр "масс" точек.
"""
function center(points, area)
    x0 = 0.0
    y0 = 0.0
    n=0
    for p in points
        if p in area
            n+=1
            x0+=p.x
            y0+=p.y
        end
    end
    return Point(x0/n, y0/n)
end

function center(points)
    x0 = 0.0
    y0 = 0.0
    n=0
    for p in points
        n+=1
        x0+=p.x
        y0+=p.y
    end
    return Point(x0/n, y0/n)
end

"""
    neighbors(points, origin, k) -> Vector{Point}

Поиск ближайших k соседей точки origin среди точек points.
"""
function neighbors(points, origin, k)
    
    if k>=length(points)
        v = []
        for p in points
            
            if LinearAlgebra.norm(p - origin)!=0.0
                push!(v, p)

            end
        end
        return v
    end
    
    if k<=0
        return [] 
    end
    
    if k>0
     distances = []
     for p in points
         d = LinearAlgebra.norm(p - origin)
         if d!=0.0
            push!(distances, d)
        end
    end

     distances = sort(distances)
     distances = distances[1:k]
     v = []
     j=0
        
     for p in points
         if LinearAlgebra.norm(p - origin)!=0.0 && (LinearAlgebra.norm(p - origin) in distances)
            push!(v, p)
            j+=1
        end
            
      if j==k break end
       end
        
    return v
    end
end
"""
    Circle(o::Point, radius)

Круг с центром o и радиусом radius.
"""
struct Circle{TO<:Point, TR<:Real}
  o::TO
  radius::TR
end

Base.:in(p::Point, c::Circle) = dist(p-c.o) <= c.radius

"""
    Square(o::Point, side)

Квадрат с центром в o и стороной side. Стороны квадрата параллельны осям координат.
"""
struct Square{TO<:Point, TS<:Real}
  o::TO
  side::TS
end

Base.:in(p::Point, s::Square) = abs((s.o).x-p.x) <= s.side/2 && abs((s.o).y-p.y) <= s.side/2

"""
    center(points, area) -> Point

Центр масс точек points, принадлежащих области area (Circle или Square).  
"""
center(points, area)


center

In [38]:
#using .Points1

println(Point(1, 2) - Point(3.0, 4.1))
println(Point(1, 2)*2)
#println(2/Point(1, 2))
neighbors([Point(1.0, 2.0), Point(1.0, 3.0), Point(1.0, 4.0), Point(0.0, 0.0),], Point(1,0), 2)

Point(-2.0, -2.0999999999999996)
Point(2, 4)


LoadError: ArgumentError: invalid index: (1.0, 4) of type Tuple{Float64, Int64}

In [13]:
using ScoredTests
#using Points
using Test
import LinearAlgebra


const AWARD1 = 1
const AWARD2 = 2

ScoredTests.DefaultScoring.award = AWARD1
ScoredTests.DefaultScoring.penalty = 0


maints = ScoredTestSet("Реализация модуля Points")

# Конструкторы struct Point
maints *= let ts = ScoredTestSet("Конструкторы struct Point")
    ts *= @scoredtest Point(1, 2) isa Point name="Point(::Int, ::Int)"
    ts *= @scoredtest Point(1.0, 2.0) isa Point name="Point(::Float64, ::Float64)"
    ts *= @scoredtest Point(1, 2.5) isa Point name="Point(::Int, ::Float64)"
    ts *= @scoredtest Point(1.0, 2) isa Point name="Point(::Float64, ::Int)"
    ts *= @scoredtest Point(1//1, 2) isa Point name="Point(::Rational, Int)" award=AWARD2
end

# Линейные операции
maints *= let ts = ScoredTestSet("Линейные операции")
    ts *= @scoredtest Point(1, 2) + Point(4, 5) == Point(5, 7) name="p + q"
    ts *= @scoredtest Point(1, 2) - Point(4, 5) == Point(-3, -3) name="p - q"
    ts *= @scoredtest -Point(1, 2) == Point(-1, -2) name="-p"
    ts *= @scoredtest 2.5 * Point(1, 2) == Point(2.5, 5.0) name="α * p"
    ts *= @scoredtest -2.5 * Point(1, 2) == Point(-2.5, -5.0) name="α * p"
    ts *= @scoredtest Point(1, 2) * 2.5 == Point(2.5, 5.0) name="p * α"
    ts *= @scoredtest Point(1, 2) / 2 == Point(0.5, 1.0) name="p / α"
end

maints *= let ts = ScoredTestSet("Расширения для LinearAlgebra")
    ts *= @scoredtest LinearAlgebra.norm(Point(1, 1)) ≈ √2 name="Норма"
    ts *= @scoredtest LinearAlgebra.dot(Point(1, 2), Point(2, 3)) == 8 name="Скалярное произведение"
end

maints *= let ts = ScoredTestSet("Конструкторы Circle")
    ts *= @scoredtest Circle(Point(0, 0), 2) isa Circle name="Circle(::Point{Int}, ::Int)"
    ts *= @scoredtest Circle(Point(0, 0), 2.0) isa Circle name="Circle(::Point{Int}, ::Float64)" award=AWARD2
end

maints *= let ts = ScoredTestSet("Точка в круге (расширение Base)")
    ts *= @scoredtest Point(1, 1) in Circle(Point(0, 0), 2)
    ts *= @scoredtest Point(1, 0) in Circle(Point(2.0, 0.0), nextfloat(1.0))
end

maints *= let ts = ScoredTestSet("Конструкторы Square")
    ts *= @scoredtest Square(Point(0, 0), 2) isa Square name="Square(::Point{Int}, ::Int)"
    ts *= @scoredtest Square(Point(0, 0), 2.0) isa Square name="Square(::Point{Int}, ::Float64)" award=AWARD2
end

maints *= let ts = ScoredTestSet("Точка в квадрате (расширение Base)")
    ts *= @scoredtest Point(1, 1) in Square(Point(0, 0), 2)
    ts *= @scoredtest !(Point(1, 1) in Square(Point(2.0, 2.0), prevfloat(2.0)))
end

maints *= let ts = ScoredTestSet("Центр масс")
    ts *= @scoredtest center(i * Point(1, 1) for i in 1:10) == Point(5.5, 5.5) name="Без указания области" award=AWARD2

    try
        points = [Point(x, y) for x in -2:2, y in -2:2]
        s = Square(Point(0, 0) + Point(0.5, 0.5), 2.0)
        ts *= @scoredtest center(points, Circle(Point(1, 2), 1)) == Point(1.0, 1.75) name="Лежащие в круге" award=AWARD2
        ts *= @scoredtest center(points, s) == Point(0.5, 0.5) name="Лежащие в квадрате" award=AWARD2
    catch
        ts *= @scoredtest error("Не реализованы операции") name="Лежащие в круге"
        ts *= @scoredtest error("Не реализованы операции") name="Лежащие в квадрате"
    end
    ts
end

maints *= let ts = ScoredTestSet("k-Ближайших соседей")
    try
        points = [Point(x, y) for x in -2:2, y in -2:2]
        origin = Point(0, 0)
        ts *= @scoredtest isempty(neighbors(points, origin, 0)) award=AWARD2
        ts *= @scoredtest isempty(neighbors(points, origin, -1)) award=AWARD2

        n4 = Set(Point(x, y) for x in -1:1, y in -1:1 if  abs(x + y) == 1)
        ts *= @scoredtest first(neighbors(points, origin, 1)) in n4 award=AWARD2
        ts *= @scoredtest all(x -> x in n4, neighbors(points, origin, 4)) award=AWARD2

        ts *= @scoredtest all(x -> x in setdiff(Set(points[2:4, 2:4]), [origin]), neighbors(points, origin, 8)) award=AWARD2
        ts *= @scoredtest Set(neighbors(points, origin, 2 * length(points))) == setdiff(Set(points), [origin]) award=AWARD2
    catch
        for _ in 1:6
            ts *= @scoredtest error("Не реализованы операции") award=AWARD2
        end
    end
    ts
end

maints *= let ts = ScoredTestSet("Дополнительно")
    try
        points = [Point(x, y) for x in -2:2, y in -2:2]
        origin = points[3, 3]
        try
            for k in 1:length(points)+1
                @inferred neighbors(points, origin, k)
            end
            ts *= @scoredtest true award=4 name="Стабильность функции neighbors"
        catch
            ts *= @scoredtest false name="Стабильность функции neighbors"
        end

        try
            @inferred center(points, Circle(Point(1, 2), 1))
            ts *= @scoredtest true award=AWARD2 name="Стабильность функции center для круга"
        catch
            ts *= @scoredtest false name="Стабильность функции center для круга"
        end

        try
            @inferred center(points, Square(Point(0, 0) + Point(0.5, 0.5), 2.0))
            ts *= @scoredtest true award=AWARD2 name="Стабильность функции center для квадрата"
        catch
            ts *= @scoredtest false name="Стабильность функции center для квадрата"
        end
    catch
        ts *= @scoredtest error("Не реализованы операции") award=4
        ts *= @scoredtest error("Не реализованы операции") award=AWARD2
        ts *= @scoredtest error("Не реализованы операции") award=AWARD2
    end
    ts
end

printsummary(maints)

Results
Реализация модуля Points

Конструкторы struct Point
No.   Result  Score  [Name]  [Error]
   1       ✓      1  Point(::Int, ::Int)
   2       ✓      1  Point(::Float64, ::Float64)
   3       ✓      1  Point(::Int, ::Float64)
   4       ✓      1  Point(::Float64, ::Int)
   5       ✓      2  Point(::Rational, Int)

Линейные операции
No.   Result  Score  [Name]  [Error]
   6       ✓      1  p + q
   7       ✓      1  p - q
   8       ✓      1  -p
   9       ✓      1  α * p
  10       ✓      1  α * p
  11       ✓      1  p * α
  12       ✓      1  p / α

Расширения для LinearAlgebra
No.   Result  Score  [Name]  [Error]
  13       ✓      1  Норма
  14       ✓      1  Скалярное произведение

Конструкторы Circle
No.   Result  Score  [Name]  [Error]
  15       ✓      1  Circle(::Point{Int}, ::Int)
  16       ✓      2  Circle(::Point{Int}, ::Float64)

Точка в круге (расширение Base)
No.   Result  Score  [Name]  [Error]
  17       ✓      1
  18       ✓      1

Конструкторы Square
No.   Re

In [3]:
  26       E      0  Error occured: MethodError(neighbors, (Point[Point(-2, -2) Point(-2, -1) Point(-2, 0) Point(-2, 1) Point(-2, 2); Point(-1, -2) Point(-1, -1) Point(-1, 0) Point(-1, 1) Point(-1, 2); Point(0, -2) Point(0, -1) Point(0, 0) Point(0, 1) Point(0, 2); Point(1, -2) Point(1, -1) Point(1, 0) Point(1, 1) Point(1, 2); Point(2, -2) Point(2, -1) Point(2, 0) Point(2, 1) Point(2, 2)], Point(0, 0), 0), 0x0000000000007ee6)
  27       E      0  Error occured: MethodError(neighbors, (Point[Point(-2, -2) Point(-2, -1) Point(-2, 0) Point(-2, 1) Point(-2, 2); Point(-1, -2) Point(-1, -1) Point(-1, 0) Point(-1, 1) Point(-1, 2); Point(0, -2) Point(0, -1) Point(0, 0) Point(0, 1) Point(0, 2); Point(1, -2) Point(1, -1) Point(1, 0) Point(1, 1) Point(1, 2); Point(2, -2) Point(2, -1) Point(2, 0) Point(2, 1) Point(2, 2)], Point(0, 0), -1), 0x0000000000007ee6)
  28       E      0  Error occured: MethodError(neighbors, (Point[Point(-2, -2) Point(-2, -1) Point(-2, 0) Point(-2, 1) Point(-2, 2); Point(-1, -2) Point(-1, -1) Point(-1, 0) Point(-1, 1) Point(-1, 2); Point(0, -2) Point(0, -1) Point(0, 0) Point(0, 1) Point(0, 2); Point(1, -2) Point(1, -1) Point(1, 0) Point(1, 1) Point(1, 2); Point(2, -2) Point(2, -1) Point(2, 0) Point(2, 1) Point(2, 2)], Point(0, 0), 1), 0x0000000000007ee6)
  29       E      0  Error occured: MethodError(neighbors, (Point[Point(-2, -2) Point(-2, -1) Point(-2, 0) Point(-2, 1) Point(-2, 2); Point(-1, -2) Point(-1, -1) Point(-1, 0) Point(-1, 1) Point(-1, 2); Point(0, -2) Point(0, -1) Point(0, 0) Point(0, 1) Point(0, 2); Point(1, -2) Point(1, -1) Point(1, 0) Point(1, 1) Point(1, 2); Point(2, -2) Point(2, -1) Point(2, 0) Point(2, 1) Point(2, 2)], Point(0, 0), 4), 0x0000000000007ee6)
  30       E      0  Error occured: MethodError(neighbors, (Point[Point(-2, -2) Point(-2, -1) Point(-2, 0) Point(-2, 1) Point(-2, 2); Point(-1, -2) Point(-1, -1) Point(-1, 0) Point(-1, 1) Point(-1, 2); Point(0, -2) Point(0, -1) Point(0, 0) Point(0, 1) Point(0, 2); Point(1, -2) Point(1, -1) Point(1, 0) Point(1, 1) Point(1, 2); Point(2, -2) Point(2, -1) Point(2, 0) Point(2, 1) Point(2, 2)], Point(0, 0), 8), 0x0000000000007ee6)
  31       E      0  Error occured: MethodError(neighbors, (Point[Point(-2, -2) Point(-2, -1) Point(-2, 0) Point(-2, 1) Point(-2, 2); Point(-1, -2) Point(-1, -1) Point(-1, 0) Point(-1, 1) Point(-1, 2); Point(0, -2) Point(0, -1) Point(0, 0) Point(0, 1) Point(0, 2); Point(1, -2) Point(1, -1) Point(1, 0) Point(1, 1) Point(1, 2); Point(2, -2) Point(2, -1) Point(2, 0) Point(2, 1) Point(2, 2)], Point(0, 0), 50), 0x0000000000007ee6)

LoadError: syntax: extra token "E" after end of expression